In [ ]:
import pandas as pd
import numpy as np
import warnings
import sys

from pathlib import Path
from scipy import stats

warnings.filterwarnings('ignore')

In [66]:
# Data split: 80% dev (70% train, 10% val), 20% test
# Scaling: Fit on training data, apply to train/val/test
sys.path.insert(0, '/media/vanafa/1TB/Workspace/Cuatri 6/ML/TP Final')

DATA_DIR = Path('/media/vanafa/1TB/Workspace/Cuatri 6/ML/TP Final/Data')
csv_files = list(Path(DATA_DIR / 'VED_DynamicData').glob('*.csv'))

for f in csv_files:
    df = pd.read_csv(f, nrows=5)
print('Columnas dynamic len:', len(df.columns))

Columnas dynamic len: 22


In [67]:
# archivo dinámico
f = csv_files[0]
df0 = pd.read_csv(f, nrows=1000)
display(df0.head())
print('Dtypes:')
print(df0.dtypes)

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],MAF[g/sec],Engine RPM[RPM],Absolute Load[%],...,Air Conditioning Power[kW],Air Conditioning Power[Watts],Heater Power[Watts],HV Battery Current[A],HV Battery SOC[%],HV Battery Voltage[V],Short Term Fuel Trim Bank 1[%],Short Term Fuel Trim Bank 2[%],Long Term Fuel Trim Bank 1[%],Long Term Fuel Trim Bank 2[%]
0,113.540988,11,2053,0,42.285918,-83.745593,26.799999,0.87,0,NaN,...,NaN,0.0,NaN,-19.400024,76.911385,317.0,NaN,NaN,NaN,NaN
1,113.540988,11,2053,1000,42.285918,-83.745593,28.920000,0.87,0,NaN,...,NaN,0.0,NaN,-110.950012,76.911385,317.0,NaN,NaN,NaN,NaN
2,113.540988,11,2053,1400,42.285918,-83.745593,28.920000,0.87,0,NaN,...,NaN,0.0,NaN,-110.950012,76.911385,317.0,NaN,NaN,NaN,NaN
3,113.540988,11,2053,2000,42.285918,-83.745593,33.200001,0.87,0,NaN,...,NaN,0.0,NaN,-109.750000,76.911385,317.0,NaN,NaN,NaN,NaN
4,113.540988,11,2053,3000,42.285740,-83.745671,36.849998,0.87,0,NaN,...,NaN,0.0,NaN,-114.650024,76.911385,317.0,NaN,NaN,NaN,NaN


Dtypes:
DayNum                            float64
VehId                               int64
Trip                                int64
Timestamp(ms)                       int64
Latitude[deg]                     float64
Longitude[deg]                    float64
Vehicle Speed[km/h]               float64
MAF[g/sec]                        float64
Engine RPM[RPM]                     int64
Absolute Load[%]                  float64
OAT[DegC]                         float64
Fuel Rate[L/hr]                   float64
Air Conditioning Power[kW]        float64
Air Conditioning Power[Watts]     float64
Heater Power[Watts]               float64
HV Battery Current[A]             float64
HV Battery SOC[%]                 float64
HV Battery Voltage[V]             float64
Short Term Fuel Trim Bank 1[%]    float64
Short Term Fuel Trim Bank 2[%]    float64
Long Term Fuel Trim Bank 1[%]     float64
Long Term Fuel Trim Bank 2[%]     float64
dtype: object


In [68]:
total_rows = 0
for f in csv_files:
    with open(f, 'rb') as fh:
        cnt = 0
        for _ in fh:
            cnt += 1
    r = max(0, cnt - 1)
    total_rows += r
print('Samples:', total_rows)

Samples: 22436808


In [69]:
dir_path = DATA_DIR / 'VED_DynamicData'

# Ensure csv_files is a list of paths
if not isinstance(csv_files, list):
    csv_files = list(Path(DATA_DIR / 'VED_DynamicData').glob('*.csv'))

files = sorted([str(p) for p in csv_files if str(dir_path) in str(p)])
if not files:
    raise FileNotFoundError(f'No files found in {dir_path}')
first_file = files[0]
print('First file:', first_file)
df_first = pd.read_csv(first_file, nrows=20)
display(df_first.head())

First file: /media/vanafa/1TB/Workspace/Cuatri 6/ML/TP Final/Data/VED_DynamicData/VED_171101_week.csv


,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],MAF[g/sec],Engine RPM[RPM],Absolute Load[%],...,Air Conditioning Power[kW],Air Conditioning Power[Watts],Heater Power[Watts],HV Battery Current[A],HV Battery SOC[%],HV Battery Voltage[V],Short Term Fuel Trim Bank 1[%],Short Term Fuel Trim Bank 2[%],Long Term Fuel Trim Bank 1[%],Long Term Fuel Trim Bank 2[%]
0,1.586651,8,706,0,42.277558,-83.69875,40,22.129999,2285,49.019608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.586651,8,706,200,42.277558,-83.69875,40,22.129999,2285,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.586651,8,706,1100,42.277558,-83.69875,45,22.129999,2285,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.586651,8,706,2100,42.277558,-83.69875,47,6.150000,2744,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.586651,8,706,4200,42.277558,-83.69875,48,21.440001,1982,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get all VED_DynamicData files
dir_path = DATA_DIR / 'VED_DynamicData'

if not isinstance(csv_files, list):
    csv_files = list(Path(DATA_DIR / 'VED_DynamicData').glob('*.csv'))

dynamic_files = sorted([str(p) for p in csv_files if str(dir_path) in str(p)])

def calculate_metrics(series):

    # Replace NaN with 0
    series = series.fillna(0)
    
    metrics = {
        'mean': series.mean(),
        'median': series.median(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        'q25': series.quantile(0.25),
        'q75': series.quantile(0.75),
        'sum': series.sum(),
        'count': len(series),
    }
    
    try:
        mode_result = stats.mode(series, keepdims=True)
        metrics['mode'] = mode_result.mode[0]
        metrics['mode_count'] = mode_result.count[0]
    except:
        metrics['mode'] = np.nan
        metrics['mode_count'] = 0
    
    metrics['range'] = metrics['max'] - metrics['min']
    metrics['iqr'] = metrics['q75'] - metrics['q25']
    
    if metrics['mean'] != 0:
        metrics['cv'] = metrics['std'] / abs(metrics['mean'])
    else:
        metrics['cv'] = 0
    
    return metrics

# Columns to skip (identifiers - keep as-is, not calculate metrics)
IDENTIFIER_COLUMNS = {'VehId', 'DayNum', 'Trip', 'Timestamp', 'Timestamp(ms)', 'Latitude[deg]', 'Longitude[deg]'}

all_results = []

for file_path in dynamic_files:
    file_name = Path(file_path).stem

    df = pd.read_csv(file_path)
    
    row_data = {'filename': file_name}
    
    # Preserve VehId and other identifiers as-is (just take first value since same for all rows)
    for id_col in IDENTIFIER_COLUMNS:
        if id_col in df.columns:
            row_data[id_col] = df[id_col].iloc[0]
    
    # Calculate metrics only for actual feature columns
    for col in df.columns:
        if col not in IDENTIFIER_COLUMNS:
            metrics = calculate_metrics(df[col])
            
            for metric_name, metric_value in metrics.items():
                key = f'{col}_{metric_name}'
                row_data[key] = metric_value
        
    all_results.append(row_data)
        
df_metrics = pd.DataFrame(all_results)

print(f'Columns: {len(df_metrics.columns)}')
print(f'VehId preserved (unaltered): {"VehId" in df_metrics.columns}')
print(df_metrics.head())


Columns: 231
VehId preserved (unaltered): True
          filename     DayNum  Latitude[deg]  Trip  Timestamp(ms)  \
0  VED_171101_week   1.586651      42.277558   706              0   
1  VED_171108_week   8.560162      42.266570   730              0   
2  VED_171115_week  15.854020      42.287548  1625              0   
3  VED_171122_week  22.501284      42.230607  1366              0   
4  VED_171129_week  29.572386      42.266449   805              0   

   Longitude[deg]  VehId  Vehicle Speed[km/h]_mean  \
0      -83.698750      8                 37.922803   
1      -83.707059      8                 39.179658   
2      -83.719066     10                 39.609396   
3      -83.680033      5                 39.195471   
4      -83.706693      8                 40.897643   

   Vehicle Speed[km/h]_median  Vehicle Speed[km/h]_std  ...  \
0                    40.00000                26.670385  ...   
1                    42.00000                27.237791  ...   
2                    42.

In [71]:
# Save results to CSV
output_file = DATA_DIR / 'VED_DynamicData_Metrics.csv'
df_metrics.to_csv(output_file, index=False)
print(f'Shape: {df_metrics.shape}')
print(f'Columns: {df_metrics.shape[1]}')

# Display summary info
print(f'Number of files processed: {len(df_metrics)}')
print(f'Number of features in original data: {len(dynamic_files) > 0 and len(pd.read_csv(dynamic_files[0]).columns) or "N/A"}')
print(f'Total columns in output CSV: {df_metrics.shape[1]}')
display(df_metrics.head())


Shape: (54, 231)
Columns: 231
Number of files processed: 54
Number of features in original data: 22
Total columns in output CSV: 231
Number of features in original data: 22
Total columns in output CSV: 231


,filename,DayNum,Latitude[deg],Trip,Timestamp(ms),Longitude[deg],VehId,Vehicle Speed[km/h]_mean,Vehicle Speed[km/h]_median,Vehicle Speed[km/h]_std,...,Long Term Fuel Trim Bank 2[%]_max,Long Term Fuel Trim Bank 2[%]_q25,Long Term Fuel Trim Bank 2[%]_q75,Long Term Fuel Trim Bank 2[%]_sum,Long Term Fuel Trim Bank 2[%]_count,Long Term Fuel Trim Bank 2[%]_mode,Long Term Fuel Trim Bank 2[%]_mode_count,Long Term Fuel Trim Bank 2[%]_range,Long Term Fuel Trim Bank 2[%]_iqr,Long Term Fuel Trim Bank 2[%]_cv
0,VED_171101_week,1.586651,42.277558,706,0,-83.698750,8,37.922803,40.00000,26.670385,...,10.15625,0.0,0.0,39605.46875,489414,0.0,467322,23.43750,0.0,11.751065
1,VED_171108_week,8.560162,42.266570,730,0,-83.707059,8,39.179658,42.00000,27.237791,...,25.00000,0.0,0.0,181728.90625,535198,0.0,437407,50.78125,0.0,7.342588
2,VED_171115_week,15.854020,42.287548,1625,0,-83.719066,10,39.609396,42.00000,27.312963,...,25.00000,0.0,0.0,244036.71875,602092,0.0,454136,46.09375,0.0,6.724574
3,VED_171122_week,22.501284,42.230607,1366,0,-83.680033,5,39.195471,42.00000,27.084772,...,25.00000,0.0,0.0,197303.90625,474329,0.0,360271,55.46875,0.0,6.625645
4,VED_171129_week,29.572386,42.266449,805,0,-83.706693,8,40.897643,43.03125,27.694109,...,25.00000,0.0,0.0,170446.09375,531856,0.0,407433,45.31250,0.0,8.636688


In [74]:
all_cols = df_metrics.columns.tolist()
all_cols.remove('filename')

# Remove identifier columns that don't have metrics
identifier_cols = {'VehId', 'DayNum', 'Trip', 'Timestamp', 'Timestamp(ms)', 'Latitude[deg]', 'Longitude[deg]'}
all_cols = [col for col in all_cols if col not in identifier_cols]

# Find a sample feature to get metrics
if all_cols:
    sample_feature_cols = [col for col in all_cols if col.startswith(all_cols[0].split('_')[0])]
    print(f'\nFeature: "{sample_feature_cols[0].rsplit("_", 1)[0]}"')
    print(f'Available metrics:')
    for col in sample_feature_cols:
        metric = col.rsplit('_', 1)[1]
        print(f'  - {metric}')

    print(f'\nTotal unique metrics per feature: {len(sample_feature_cols)}')
    print(f'Number of original features: 22')
    print(f'Total output columns: 1 (filename) + 22 features × {len(sample_feature_cols)} metrics = {1 + 22 * len(sample_feature_cols)} columns')
else:
    print("No feature columns found!")



Feature: "Vehicle Speed[km/h]"
Available metrics:
  - mean
  - median
  - std
  - min
  - max
  - q25
  - q75
  - sum
  - count
  - mode
  - count
  - range
  - iqr
  - cv

Total unique metrics per feature: 14
Number of original features: 22
Total output columns: 1 (filename) + 22 features × 14 metrics = 309 columns


In [ ]:
# Fourier Analysis for Dynamic Data Files
EXCLUDE_COLUMNS = {
    'DayNum', 'Trip', 'Timestamp(ms)', 
    'Latitude[deg]', 'Longitude[deg]'
    # Note: VehId is intentionally excluded to preserve it for identification
}

def apply_fourier_analysis(series, n_components=10):
    series = series.fillna(0).values
    
    fft_result = np.fft.fft(series)
    magnitude = np.abs(fft_result)
    frequencies = np.fft.fftfreq(len(series))
    
    n_samples = len(series)
    positive_freq_end = n_samples // 2
    
    mag_positive = magnitude[1:positive_freq_end]
    freq_positive = frequencies[1:positive_freq_end]
    
    sorted_indices = np.argsort(mag_positive)[::-1]
    
    n_comp = min(n_components, len(mag_positive))
    top_indices = sorted_indices[:n_comp]
    
    result = {}
    for i in range(n_comp):
        idx = top_indices[i]
        result[f'fft_magnitude_{i}'] = mag_positive[idx]
        result[f'fft_frequency_{i}'] = freq_positive[idx]
    
    result['fft_energy'] = np.sum(magnitude ** 2)
    
    if np.sum(mag_positive) > 0:
        result['fft_spectral_centroid'] = np.sum(freq_positive * mag_positive) / np.sum(mag_positive)
    else:
        result['fft_spectral_centroid'] = 0
    
    return result

fourier_results = []
veh_ids_fourier = []  # Store VehIds from the raw data
n_fft_components = 10

for file_path in dynamic_files:
    file_name = Path(file_path).stem
    print(f'Processing with Fourier: {file_name}')

    df = pd.read_csv(file_path)
    row_data = {'filename': file_name}
    
    # Extract VehId if available in the raw data
    if 'VehId' in df.columns:
        # Get the first VehId value (same for all rows in file)
        veh_id = df['VehId'].iloc[0]
        row_data['VehId'] = veh_id
        veh_ids_fourier.append(veh_id)
    
    for col in df.columns:
        if col not in EXCLUDE_COLUMNS and col != 'VehId':
            fft_metrics = apply_fourier_analysis(df[col], n_components=n_fft_components)
            
            for metric_name, metric_value in fft_metrics.items():
                key = f'{col}_{metric_name}'
                row_data[key] = metric_value
    
    fourier_results.append(row_data)
        
df_fourier = pd.DataFrame(fourier_results)

print(f'\nshape: {df_fourier.shape}')
print(f'Columns: {len(df_fourier.columns)}')
print(f'Features with Fourier analysis: {len([c for c in df.columns if c not in EXCLUDE_COLUMNS and c != "VehId"])}')
print(f'VehId preserved: {"VehId" in df_fourier.columns}')
print(df_fourier.head())


Processing with Fourier: VED_171101_week
Processing with Fourier: VED_171108_week
Processing with Fourier: VED_171108_week
Processing with Fourier: VED_171115_week
Processing with Fourier: VED_171115_week
Processing with Fourier: VED_171122_week
Processing with Fourier: VED_171122_week
Processing with Fourier: VED_171129_week
Processing with Fourier: VED_171129_week
Processing with Fourier: VED_171206_week
Processing with Fourier: VED_171206_week
Processing with Fourier: VED_171213_week
Processing with Fourier: VED_171213_week
Processing with Fourier: VED_171220_week
Processing with Fourier: VED_171220_week
Processing with Fourier: VED_171227_week
Processing with Fourier: VED_171227_week
Processing with Fourier: VED_180103_week
Processing with Fourier: VED_180103_week
Processing with Fourier: VED_180110_week
Processing with Fourier: VED_180110_week
Processing with Fourier: VED_180117_week
Processing with Fourier: VED_180117_week
Processing with Fourier: VED_180124_week
Processing with 

In [ ]:
# Save Fourier results to CSV
output_file_fourier = DATA_DIR / 'VED_DynamicData_Fourier.csv'
df_fourier.to_csv(output_file_fourier, index=False)
print(f'Shape: {df_fourier.shape}')
print(f'Rows (samples): {df_fourier.shape[0]}')
print(f'Columns: {df_fourier.shape[1]}')

print(f'Number of files processed: {len(df_fourier)}')
print(f'Number of features analyzed: {(df_fourier.shape[1] - 2) // 22 if "VehId" in df_fourier.columns else (df_fourier.shape[1] - 1) // 22}')
print(f'Top frequency components per feature: 10')
print(f'Additional metrics per feature: energy, spectral_centroid')
print(f'VehId preserved for vehicle identification: {"VehId" in df_fourier.columns}')

print(f'\nFirst few rows:')
display(df_fourier.head())


Shape: (54, 354)
Rows (samples): 54
Columns: 354
Number of files processed: 54
Number of features analyzed: 16
Top frequency components per feature: 10
Additional metrics per feature: energy, spectral_centroid
VehId preserved for vehicle identification: True

First few rows:


,filename,VehId,Vehicle Speed[km/h]_fft_magnitude_0,Vehicle Speed[km/h]_fft_frequency_0,Vehicle Speed[km/h]_fft_magnitude_1,Vehicle Speed[km/h]_fft_frequency_1,Vehicle Speed[km/h]_fft_magnitude_2,Vehicle Speed[km/h]_fft_frequency_2,Vehicle Speed[km/h]_fft_magnitude_3,Vehicle Speed[km/h]_fft_frequency_3,...,Long Term Fuel Trim Bank 2[%]_fft_magnitude_6,Long Term Fuel Trim Bank 2[%]_fft_frequency_6,Long Term Fuel Trim Bank 2[%]_fft_magnitude_7,Long Term Fuel Trim Bank 2[%]_fft_frequency_7,Long Term Fuel Trim Bank 2[%]_fft_magnitude_8,Long Term Fuel Trim Bank 2[%]_fft_frequency_8,Long Term Fuel Trim Bank 2[%]_fft_magnitude_9,Long Term Fuel Trim Bank 2[%]_fft_frequency_9,Long Term Fuel Trim Bank 2[%]_fft_energy,Long Term Fuel Trim Bank 2[%]_fft_spectral_centroid
0,VED_171101_week,8,892401.478753,0.000072,664643.291298,0.000033,623817.222020,0.000002,615634.096753,0.000135,...,37015.938060,0.000035,35018.211231,0.000153,32911.542443,0.000025,32519.286730,0.000106,2.181713e+11,0.086314
1,VED_171108_week,8,765811.893671,0.000245,735155.596969,0.000002,714534.448518,0.000052,684102.285574,0.000007,...,95027.490148,0.000024,93433.839763,0.000071,92193.439538,0.000108,88273.993931,0.000037,1.813540e+12,0.085907
2,VED_171115_week,10,871365.320250,0.000050,850495.485345,0.000030,813922.686091,0.000012,762820.148156,0.000721,...,125365.875172,0.000126,119782.795748,0.000081,118594.136382,0.000138,116575.361426,0.000080,2.752571e+12,0.087781
3,VED_171122_week,5,652460.291133,0.000118,639900.903583,0.000156,633348.580358,0.000002,619041.564168,0.000011,...,87461.235095,0.000053,86780.612535,0.000173,86270.026104,0.000008,85989.410944,0.000211,1.747868e+12,0.085821
4,VED_171129_week,8,730041.191683,0.000199,716313.722582,0.000096,677605.414804,0.000034,663804.703289,0.000038,...,120711.283273,0.000070,118442.619175,0.000030,116406.974024,0.000047,115480.477705,0.000079,2.196096e+12,0.085544
